# FeatureTools

Featuretools is an open-source Python library designed for automated feature engineering. It was developed by the Feature Labs. It enables the creation of new features from several related data tables. Feature selection techniques can then be used to choose appropriate features from them and then data scientists can proceed with model creation.

To read about it more, please read [this](https://analyticsindiamag.com/introduction-to-featuretools-a-python-framework-for-automated-feature-engineering/) article.

# Practical implementation 

Here’s a demonstration of implementing automated feature engineering using Featuretools for a supervised machine learning classification task that aims to predict whether or not a loan application of a financial institution named ‘Home Credit’ will default on the loan (‘Default’ means the client fails to repay the loan). The ‘Home Credit Default Risk’ dataset used here is available on Kaggle ([weblink](https://www.kaggle.com/c/home-credit-default-risk/data) to download it). Step-wise explanation of the code is as follows:

In [ ]:
!python -m pip install pip --upgrade --user -q
!python -m pip install numpy pandas seaborn matplotlib scipy sklearn statsmodels tensorflow keras --user -q

  Install Featuretools library

In [ ]:
!python -m pip install featuretools --user -q

In [ ]:
import IPython
IPython.Application.instance().kernel.do_shutdown(True)

Import required libraries and modules

In [ ]:
import numpy as np
import pandas as pd
import featuretools as ft
import featuretools.variable_types as vtypes 

Read the data files

In [ ]:
#Files containing training and test data for each client
app_train = pd.read_csv('https://gitlab.com/AnalyticsIndiaMagazine/practicedatasets/-/raw/main/featuretools/application_train.csv').replace({365243: np.nan})
app_test = pd.read_csv('https://gitlab.com/AnalyticsIndiaMagazine/practicedatasets/-/raw/main/featuretools/application_test.csv').replace({365243: np.nan})

#File containing data of clients’ previous credits from financial institutions other than Home Credit

bureau = pd.read_csv('https://gitlab.com/AnalyticsIndiaMagazine/practicedatasets/-/raw/main/featuretools/bureau.csv').replace({365243: np.nan})
#File containing data about monthly balance about the credits
bureau_balance = pd.read_csv('https://gitlab.com/AnalyticsIndiaMagazine/practicedatasets/-/raw/main/featuretools/bureau_balance.csv').replace({365243: np.nan})
#File containing monthly data about cash loans or previous point of sale from previous loan data 
cash = pd.read_csv('https://gitlab.com/AnalyticsIndiaMagazine/practicedatasets/-/raw/main/featuretools/POS_CASH_balance.csv').replace({365243: np.nan})
#File containing data regarding previous credit card loans
credit = pd.read_csv('https://media.githubusercontent.com/media/analyticsindiamagazine/PracticeDatasets/main/featuretools/credit_card_balance.csv').replace({365243: np.nan})
#File having data of previous loan applications at Home Credit
previous = pd.read_csv('https://gitlab.com/AnalyticsIndiaMagazine/practicedatasets/-/raw/main/featuretools/previous_application.csv').replace({365243: np.nan})
#File containing data about payment history for Home Credit’s previous loans
installments = pd.read_csv('https://gitlab.com/AnalyticsIndiaMagazine/practicedatasets/-/raw/main/featuretools/installments_payments.csv').replace({365243: np.nan}) 

Join the training set and test set so that same features can be built for both

In [ ]:
#Create a target column in test set before merging
app_test['TARGET'] = np.nan
# Append test set to the training set
app = app_train.append(app_test, ignore_index = True, sort = True) 

  Convert floating point indexes to integer type for adding relationships

In [ ]:
#for each floating point index
for index in ['SK_ID_CURR', 'SK_ID_PREV', 'SK_ID_BUREAU']:
  #for each of the dataframes
    for dataset in [app, bureau, bureau_balance, cash, credit, 
    previous, installments]:
    #convert if the index is one of the columns of the dataframe
        if index in list(dataset.columns):
  #Fill null records with 0 and change datatype to integer
            dataset[index] = dataset[index].fillna(0).astype(np.int64)

  Identify the boolean variables which are recorded as integers (0.0 or 1.0)

In [ ]:
# Entity set with id applications
es = ft.EntitySet(id = 'clients')

In [ ]:
#Create a list to specify boolean type for ‘app’ data
app_types = {}
# For each column in the dataset
for col in app:
#if the column has 2 unique values and is of numeric type 
    if (app[col].nunique() == 2) and (app[col].dtype == float):
    #assign the type as Boolean 
        app_types[col] = vtypes.Boolean
# Remove the TARGET member from the ‘app_types’ list
del app_types['TARGET']
#Display the number of Boolean variables
print('There are {} Boolean variables in the application data.'.format(len(app_types))) 

Assign ‘Oridinal’ datatype to the columns of ‘app’ data which can have ordered discrete values

In [ ]:
app_types['REGION_RATING_CLIENT'] = vtypes.Ordinal
app_types['REGION_RATING_CLIENT_W_CITY'] = vtypes.Ordinal
app_types['HOUR_APPR_PROCESS_START'] = vtypes.Ordinal 

As done for ‘app’ data, identify the Boolean variables of ‘previous’ data 

In [ ]:
#Create list to specify the datatype
previous_types = {}
# For each column in the ‘previous’ data
for col in previous:
#If the column has 2 unique values (0.0 or 1.0) and its datatype is numeric
    if (previous[col].nunique() == 2) and (previous[col].dtype == float):
#Assign the datatype as Boolean
        previous_types[col] = vtypes.Boolean
#Display the number of Boolean variables in the ‘previous’ data
print('There are {} Boolean variables in the previous data.'.format(len(previous_types))) 

The ‘credit’, ‘cash’ and ‘installments’ data SK_ID_CURR variable. That variable is not needed since we will link these three dataframes to the ‘app’ data through ‘previous’ data using SK_ID_PREV variable.

In [ ]:
installments = installments.drop(columns = ['SK_ID_CURR'])
credit = credit.drop(columns = ['SK_ID_CURR'])
cash = cash.drop(columns = ['SK_ID_CURR']) 

Add the seven data tables to the entity set. entity_from_dataframe() method loads the data for a specific entity from a specified dataframe.

In [ ]:
es = es.entity_from_dataframe(entity_id = 'app', dataframe = app, index = 
'SK_ID_CURR',variable_types = app_types)
es = es.entity_from_dataframe(entity_id = 'bureau', dataframe = bureau, index = 'SK_ID_BUREAU')
es = es.entity_from_dataframe(entity_id = 'previous', dataframe = previous, index = 'SK_ID_PREV',variable_types = previous_types)

#For entities which do not have a unique index, create index too by setting ‘make_index’ to True

es = es.entity_from_dataframe(entity_id = 'bureau_balance', dataframe = bureau_balance, make_index = True, index = 'bureaubalance_index')
es = es.entity_from_dataframe(entity_id = 'cash', dataframe = cash, 
                              make_index = True, index = 'cash_index')
es = es.entity_from_dataframe(entity_id = 'installments', dataframe = installments,make_index = True, index = 'installments_index')
es = es.entity_from_dataframe(entity_id = 'credit', dataframe = credit,
make_index = True, index = 'credit_index')
#Display the entityset
es 

Identify relationships: e.g. ‘app’ dataframe has a single record for each client identified by the key SK_ID_CURR. The ‘bureau’ dataframe has multiple records for each client. So ‘app’ dataframe is the parent while ‘bureau’ dataframe is the child in parent-shild relationship among tables. Display there relationship:

In [ ]:
print('Parent: app, Parent Variable of bureau: SK_ID_CURR\n\n', app.iloc[:, 111:115].head())

 The ‘buraeau’ and ‘buraeu_balance’ dataframes are linked through a shared variable called SK_ID_BUREAU. This variable is called ‘parent variable’ in the parent table bureau while ‘child variable’ in the child table bureau_balance.

In [ ]:
print('Parent: bureau, Parent Variable of bureau_balance: SK_ID_BUREAU\n\n', bureau.iloc[:, :5].head())
print('\nChild: bureau_balance, Child Variable of bureau: SK_ID_BUREAU\n\n', bureau_balance.head()) 

Add new relationships among the dataframes to be added to the entityset. Relationship class enables representing relationships between various entities. 

In [ ]:
# app_train and bureau relation
r_app_bureau = ft.Relationship(es['app']['SK_ID_CURR'], es['bureau']['SK_ID_CURR'])
# bureau and bureau_balance relation
r_bureau_balance = ft.Relationship(es['bureau']['SK_ID_BUREAU'], es['bureau_balance']['SK_ID_BUREAU'])
# current app and previous app realtion
r_app_previous = ft.Relationship(es['app']['SK_ID_CURR'], es['previous']['SK_ID_CURR'])
#cash, installments, and credit’s relation with previous app
r_previous_cash = ft.Relationship(es['previous']['SK_ID_PREV'], es['cash']['SK_ID_PREV'])
r_previous_installments = ft.Relationship(es['previous']['SK_ID_PREV'], es['installments']['SK_ID_PREV'])
r_previous_credit = ft.Relationship(es['previous']['SK_ID_PREV'], es['credit']['SK_ID_PREV']) 

  Add the above-created relationships to the entityset:

In [ ]:
es = es.add_relationships([r_app_bureau, r_bureau_balance, r_app_previous,r_previous_cash, r_previous_installments, r_previous_credit])
#Print the modified entityset
es 

  Visualize the entityset.

In [ ]:
es.plot()

Record the feature primitives 

In [ ]:
primitives = ft.list_primitives()
#Set maximum column width for displaying the primitives
pd.options.display.max_colwidth = 100
# Display the records with aggregation primitive
primitives[primitives['type'] == 'aggregation'].head(10) 


Display the records with transformation primitive.



In [ ]:
primitives[primitives['type'] == 'transform'].head(10)

 Build new features using default primitives of featuretools.

Specify the default aggregation and transformation primitives.

In [ ]:
#Aggregation primitives
default_agg_primitives =  ["sum", "std", "max", "skew", "min", "mean", "count", "percent_true", "num_unique", "mode"]
#Transformation primitives
default_trans_primitives =  ["day", "year", "month", "weekday", "haversine", "num_words", "num_characters"] 

Create new features from a list of relationships and a dictionary of entities using dfs() method.

In [ ]:
feature_names = ft.dfs(entityset = es, target_entity = 'app',
                      trans_primitives = default_trans_primitives,
                      agg_primitives=default_agg_primitives, 
                      where_primitives = [], seed_features = [],
                      max_depth = 2, n_jobs = -1, verbose = 1,
                      features_only=True) 

Display some of the newly generated features

In [ ]:
feature_names[1050:1070]